## Can trading using RSI beat buying and holding a stock?

### Background: 
The Relative Strength Index (RSI) is a momentum indicator that describes the current price relative to average high and low prices over a previous trading period. This indicator estimates overbought or oversold status and helps spot trend reversals, price pullbacks, and the emergence of bullish or bearish markets.

The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, “New Concepts in Technical Trading Systems.”

An asset is usually considered overbought when the RSI is above 70% and oversold when it is below 30%.

Here is the process of calculating RSI:

1. For 14 periods, calculate the difference in price from the current period and the previous period;
2. For each period, record each positive change in price as a gain and each negative change as a loss;
3. On the 14th period, calculate the arithmetic mean of the gains and losses for the entire 14 day period (e.g. gains / 14 & losses / 14);
4. Use these values to calculate the RS
5. Use the RS value to calculate the RSI
6. For each proceeding period, use only the previous RSI value to calculate the next average value by multiplying by our lookback period – 1 (e.g. 13 for a lookback of 14).
7. Add the value obtained in step 6 to the current day’s value (do this for both the gains and losses)

<img src="rsi.jpeg" width=300 height=300 />

### Resource:

- https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209
- https://www.tradingview.com/support/solutions/43000502338-relative-strength-index-rsi/
- https://en.wikipedia.org/wiki/Relative_strength_index

### Import Libraries and Finance Data 

In [1]:
# import basic libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import altair as alt
from IPython.display import display_html
from itertools import chain,cycle

# adjust the width of jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

#three ways to import finance data, we will mainly use yfinance 
#https://pypi.org/project/yfinance    OR  https://www.alpharithms.com/python-financial-data-491110/

# import quandl
# df = quandl.get('WIKI/NVDA')

# import pandas_datareader as pdr
# #request API call from https://data.nasdaq.com/sign-up
# df = pdr.get_data_quandl("NVDA", api_key="vjsci8v8EfV-BpoQmxbR")

import yfinance as yf

In [2]:
## Function to show dataframes side by side
## https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [3]:
# Get finance data, valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
Ticker="tsla"
df = yf.Ticker(Ticker).history(period='3y')
df=df.reset_index()
df['Date']=pd.to_datetime(df['Date'])
df=df[['Date','Open','High','Low','Close']]
df['Close']=round(df['Close'],2)

In [4]:
df.head()

,Date,Open,High,Low,Close
0,2019-05-28,38.240002,39.000000,37.570000,37.74
1,2019-05-29,37.419998,38.478001,37.007999,37.97
2,2019-05-30,37.750000,38.452000,37.403999,37.64
3,2019-05-31,37.020000,37.984001,36.820000,37.03
4,2019-06-03,37.102001,37.335999,35.397999,35.79


### Calculating RSI Value

My attempt on calculating the RSI value using simple average. But a typical RSI value is calculating using Wilder’s original smoothing method
<img src="wsm.jpeg" width=300 height=300 />

In [5]:
df['RSI']=round(ta.rsi(df['Close'], timeperiod=14),2)
df.head()

,Date,Open,High,Low,Close,RSI
0,2019-05-28,38.240002,39.000000,37.570000,37.74,NaN
1,2019-05-29,37.419998,38.478001,37.007999,37.97,NaN
2,2019-05-30,37.750000,38.452000,37.403999,37.64,NaN
3,2019-05-31,37.020000,37.984001,36.820000,37.03,NaN
4,2019-06-03,37.102001,37.335999,35.397999,35.79,NaN


In [6]:
df.drop(range(0,14),inplace=True)

In [9]:
df.reset_index(drop=True,inplace=True)

In [10]:
df.head()

,Date,Open,High,Low,Close,RSI
0,2019-06-17,43.096001,45.400002,42.854000,45.01,76.26
1,2019-06-18,45.743999,46.948002,44.512001,44.95,75.79
2,2019-06-19,45.021999,45.554001,44.212002,45.29,76.68
3,2019-06-20,44.599998,45.380001,43.270000,43.92,66.14
4,2019-06-21,43.243999,44.436001,43.099998,44.37,67.71


In [ ]:
#using the ta library, RSI values are calculated using daily close price. The default RSI period is 14 days.
df['RSI']=round(ta.rsi(df['Close'], timeperiod=14),2)

#Dropping the first 14 days without 14 days and reset index
df.drop(range(0,14),inplace=True)
df.reset_index(drop=True,inplace=True)

# Value is considered "overbought" when it exceeds 70 and "oversold" when indictator is below 30
df['Overbought']=70
df['Oversold']=30

In [ ]:
# uncomment if you want to see how RSI is calculated using Simple Averages

# df['Difference'] = df['Close'].diff()
# df['Difference'].fillna(0,inplace=True)
# df['PercentChange'] = round(df['Difference']/ df['Close']*100,2)
# df.loc[df['Difference']>0, 'Gain'] = df['Difference']
# df.loc[df['Difference']<0, 'Loss'] = -df['Difference']
# df['Gain'].fillna(0,inplace=True)
# df['Loss'].fillna(0,inplace=True)
# for i in range(0,df.shape[0]-14):
#     sumGain=0
#     sumLoss=0
#     for index, row in df[1+i:14+i].iterrows():
#         sumGain=sumGain+row['Gain']
#         sumLoss=sumLoss+row['Loss']
#     df.loc[df.index==(14+i),'AvgGain']=round(sumGain/8,2)
#     df.loc[df.index==(14+i),'SumGain']=sumGain    
#     df.loc[df.index==(14+i),'AvgLoss']=round(sumLoss/8,2)    
#     df.loc[df.index==(14+i),'SumLoss']=sumLoss    
# df['RS'] = round(df['AvgGain']/df['AvgLoss'],2)
# df['AvgGain'].fillna(0,inplace=True)
# df['AvgLoss'].fillna(0,inplace=True)
# df['RS'].fillna(0,inplace=True)
# df['RSI'] = round(100-(100/(1+df['RS'])),2)
# df=df.reset_index()
# df.drop(range(0,14),inplace=True)

In [ ]:
#Prepare of Stock graph
Stock = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(color="grey",size=15)).encode(
    x='Date:T',
    y='Close:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f')]
).properties(
    title='Stock Price',
    width=800,
    height=300
)


In [ ]:
#Plotting RSI chart with Oversold and Overbought lines
RSI = alt.Chart(df).mark_line().encode(
    x='Date:T',
    y='RSI:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    title='RSI',
    width=800,
    height=300)

Oversold = alt.Chart(df).mark_line(color= 'red').encode(
    x='Date:T',
    y='Oversold:Q'
)

Overbought = alt.Chart(df).mark_line(color= 'green').encode(
    x='Date:T',
    y='Overbought:Q'
)

RSI_Chart=RSI+Oversold+Overbought
Stock & RSI_Chart

### Setting up RSI Strategy 

In [ ]:
print('Current Start Date :',df.iloc[0]['Date'])

In [ ]:
# Set up your initial investment and time below
df = df.loc[df['Date']>='2019-04-15']
df.reset_index(drop=True,inplace=True)
df.head()

In [ ]:
#initial values
startAmount=1000
startPrice=df.iloc[0]['Close']
shares=round(startAmount/startPrice,3)

In [ ]:
# choose your sell and buy RSI threshold, you would sell when RSI hits the sell threhold and rebuy when RSI drops below buy thresold
RSI_Sell=70
RSI_Buy=60

In [ ]:
#RSI Dateframe with Initialize Info
df_rsi = pd.DataFrame(columns = ['Date','Price','Shares', 'Amount','RSI','Buy/Sell','Profit/Loss','Profit/Loss %'])
df_rsi = df_rsi.append({'Date' : df.iloc[0]['Date'], 
              'Price' : startPrice,
              'Shares' : shares, 
              'Amount' : startAmount,
              'RSI': df.iloc[0]['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0},ignore_index=True)

df_longterm = df_rsi

print('Start Price:', '${:,.2f}'.format(startPrice))
print('Start Shares: ', shares)
print('RSI Sell Signal: >=', RSI_Sell)
print('RSI Buy Signal: <', RSI_Buy)
df_rsi.head()

In [ ]:
## buy and sell stock based on RSI value
Sell_Flag=0
for index, row in df.iterrows():
    if index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0:
        Sell_Flag=1
        Sold_Amount=df_rsi.iloc[-1]['Shares']*row['Close']
        df_rsi = df_rsi.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': row['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss': round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2),
              'Profit/Loss %': round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
    elif index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1:
        Sell_Flag=0
        df_rsi = df_rsi.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : df_rsi.iloc[-1]['Amount']/row['Close'], 
              'Amount' : df_rsi.iloc[-1]['Amount'],
              'RSI': row['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0                              
                            },ignore_index=True)
        
        
# sell on the last day and calculate the final profit/loss        
if df_rsi.iloc[-1]['Buy/Sell']=='Buy':
    Sold_Amount=df.iloc[-1]['Close']*df_rsi.iloc[-1]['Shares']
    
    df_rsi = df_rsi.append({
              'Date' : df.iloc[-1]['Date'], 
              'Price' : df.iloc[-1]['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': df.iloc[-1]['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss':Sold_Amount-df_rsi.iloc[-1]['Amount'],
              'Profit/Loss %':round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
# add buy and sell threshold
df_rsi['Sell'] = RSI_Sell
df_rsi['Buy'] = RSI_Buy

In [ ]:
df_rsi

In [ ]:
Final_RSI = alt.Chart(df_rsi).mark_circle(color='Blue',size=90).encode(
    x='Date:T',
    y='RSI:Q',
    color='Buy/Sell',
        tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    width=800,
    height=300
)

Sell_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5],color= 'red').encode(
    x='Date:T',
    y='Sell:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

Buy_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5], color= 'green').encode(
    x='Date:T',
    y='Buy:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

#RSI+Final_RSI+Sell_Line+Buy_Line

In [ ]:
df_longterm = df_rsi.iloc[[0, -1]]
df_longterm.reset_index(drop=True,inplace=True)
df_longterm.at[1, 'Amount'] = df_longterm.iloc[-1]['Price']*df_longterm.iloc[0]['Shares']
df_longterm.at[1, 'Buy/Sell'] = 'Sell'
df_longterm.at[1, 'Profit/Loss'] = df_longterm.iloc[-1]['Amount']-df_longterm.iloc[0]['Amount']
df_longterm.at[1, 'Profit/Loss %'] = round(df_longterm.at[1, 'Profit/Loss']/df_longterm.iloc[0]['Amount']*100,2)

In [ ]:
Stock&(RSI+Final_RSI+Sell_Line+Buy_Line)

In [ ]:
display(df_rsi,df_longterm, titles=['RSI Strategy ','Long Term Buy&Hold'])